In this very simple example, we have a network with one input a hidden layer with 2 nodes and an output layer of 1 node. This is a regression problem where we are trying to predict the unknown function.

In [1]:
# Use torch to create tensors to store all of the numerical values, this includes the raw data
import torch
# Used to make the weight and bias tensors part of the neural network
import torch.nn as nn 
# torch.nn.functional gives us the activation functions
import torch.nn.functional as F 
# Stochastic gradient descent
from torch.optim import SGD

import numpy as np 
import pandas as pd 
import matplotlib as plt
# sns stands for Samual Norman Seaborn a character in the drama "The West Wing"
import seaborn as sns


In [ ]:
# Creating a new nearal network class
# nn.module is the basic object module parent class
class BasicNN(nn.Module):
    super().__init__() #inheriting the attribute of the parent class nn.Module 
    self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad=False) # using nn.Parameter gives us the option to optimize it
    self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad=False)
    self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad=False)

    self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad=False) 
    self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad=False)
    self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad=False)

    self.final_bias = nn.Parameter(torch.tensor(-16.0), requires_grad=False)

    def forward (self,input):#Create forward propagation function
        input_to_relu_1 = input * self.w00 + self.b00
        relu_1_output = F.relu(input_to_relu_1)
        scaled_relu_1_output = relu_1_out * self.w01 

        input_to_relu_2 = input * self.w10 + self.b10
        relu_2_output = F.relu(input_to_relu_2)
        scaled_relu_2_output = relu_2_out * self.w11 

        input_to_final_relu = scaled_relu_1_output + scaled_relu_2_output + self.final_bias

        output = F.relu(input_to_final_relu)

        return(output)


In [ ]:
input_doses = torch.linspace(start = 0)